# Delta calculator

A simple delta calculator

In [1]:
import pandas as pd
import numpy as np
import py_vollib.black_scholes.greeks.numerical as greeks
import py_vollib.black_scholes.implied_volatility as iv
import py_vollib.black_scholes as bs
from typing import Tuple

In [2]:
# parameters
spot:float = 17000
strike:float = 17000
risk_free_rate:float = 0.1
iv_range: Tuple[float, float] = (0.12,0.25)
step = 100 
de = 1 # days to expiry


In [3]:
kwargs = {
    'flag': 'c',
    'S': spot,
    'K': strike,
    't': de/360,
    'r': risk_free_rate
    
}

In [4]:
ivs = np.linspace(iv_range[0], iv_range[-1], 10)
strikes = set() 
for i in range(5):
    strikes.add(spot + i*step)
    strikes.add(spot - i*step)
    


In [5]:
calls = []
puts = []
for sigma in ivs:
    for strk in strikes:
        kwargs.update({'sigma': sigma, 'K': strk, 'flag': 'c'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        calls.append((sigma, strk, price, grk))
        kwargs.update({'flag': 'p'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        puts.append((sigma, strk, price, grk))
        

## Call option Delta 

In [6]:
df1 = pd.DataFrame(calls, columns=['iv', 'strike', 'price', 'greek']).round(4)
df1.pivot(index='iv', columns='strike', values='greek').round(4)

strike,16600,16700,16800,16900,17000,17100,17200,17300,17400
iv,,,,,,,,,
0.1200,0.9999,0.9979,0.9725,0.8364,0.5188,0.1894,0.0358,0.0033,0.0001
0.1344,0.9997,0.9947,0.9566,0.8093,0.5170,0.2162,0.0539,0.0076,0.0006
0.1489,0.9989,0.9895,0.9391,0.7856,0.5157,0.2394,0.0734,0.0143,0.0017
0.1633,0.9975,0.9823,0.9209,0.7648,0.5146,0.2595,0.0931,0.0230,0.0038
0.1778,0.9950,0.9735,0.9028,0.7467,0.5137,0.2770,0.1124,0.0334,0.0072
0.1922,0.9914,0.9633,0.8851,0.7307,0.5130,0.2924,0.1309,0.0451,0.0118
0.2067,0.9867,0.9521,0.8681,0.7165,0.5123,0.3059,0.1485,0.0576,0.0177
0.2211,0.9809,0.9403,0.8520,0.7040,0.5118,0.3179,0.1650,0.0706,0.0246
0.2356,0.9742,0.9282,0.8369,0.6928,0.5114,0.3286,0.1805,0.0837,0.0325


## Put option Delta 

In [7]:
df2 = pd.DataFrame(puts, columns=['iv', 'strike', 'price', 'greek']).round(4)
df2.pivot(index='iv', columns='strike', values='greek').round(4)

strike,16600,16700,16800,16900,17000,17100,17200,17300,17400
iv,,,,,,,,,
0.1200,-0.0001,-0.0021,-0.0275,-0.1636,-0.4812,-0.8106,-0.9642,-0.9967,-0.9999
0.1344,-0.0003,-0.0053,-0.0434,-0.1907,-0.4830,-0.7838,-0.9461,-0.9924,-0.9994
0.1489,-0.0011,-0.0105,-0.0609,-0.2144,-0.4843,-0.7606,-0.9266,-0.9857,-0.9983
0.1633,-0.0025,-0.0177,-0.0791,-0.2352,-0.4854,-0.7405,-0.9069,-0.9770,-0.9962
0.1778,-0.0050,-0.0265,-0.0972,-0.2533,-0.4863,-0.7230,-0.8876,-0.9666,-0.9928
0.1922,-0.0086,-0.0367,-0.1149,-0.2693,-0.4870,-0.7076,-0.8691,-0.9549,-0.9882
0.2067,-0.0133,-0.0479,-0.1319,-0.2835,-0.4877,-0.6941,-0.8515,-0.9424,-0.9823
0.2211,-0.0191,-0.0597,-0.1480,-0.2960,-0.4882,-0.6821,-0.8350,-0.9294,-0.9754
0.2356,-0.0258,-0.0718,-0.1631,-0.3072,-0.4886,-0.6714,-0.8195,-0.9163,-0.9675


In [8]:
calls = []
puts = []
spot_vals = np.linspace(spot*0.98,spot*1.02,10)
for sigma in ivs:
    for spt in spot_vals:
        kwargs.update({'sigma': sigma, 'S': spt, 'K':strike, 'flag': 'c'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        calls.append((sigma, spt, price, grk))
        kwargs.update({'flag': 'p'})
        price = bs.black_scholes(**kwargs)
        grk = greeks.delta(**kwargs)
        puts.append((sigma, spt, price, grk))

In [9]:
df3 = pd.DataFrame(calls, columns=['iv', 'spot', 'price', 'greek']).round(4)
df3['spot'] = df3.spot.astype(int)
df3_pivot = df3.pivot(index='iv', columns='spot', values='greek').round(4)
df3_pivot

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.1200,0.0008,0.0075,0.0428,0.1561,0.3803,0.6547,0.8638,0.9636,0.9936,0.9993
0.1344,0.0025,0.0150,0.0625,0.1837,0.3931,0.6391,0.8366,0.9454,0.9868,0.9977
0.1489,0.0056,0.0251,0.0831,0.2081,0.4035,0.6263,0.8122,0.9261,0.9776,0.9948
0.1633,0.0104,0.0372,0.1036,0.2295,0.4122,0.6158,0.7906,0.9066,0.9664,0.9903
0.1778,0.0169,0.0506,0.1234,0.2484,0.4195,0.6069,0.7714,0.8876,0.9537,0.9841
0.1922,0.0249,0.0649,0.1422,0.2651,0.4258,0.5993,0.7544,0.8693,0.9402,0.9765
0.2067,0.0341,0.0795,0.1599,0.2799,0.4312,0.5928,0.7392,0.8521,0.9262,0.9678
0.2211,0.0442,0.0941,0.1764,0.2931,0.4360,0.5871,0.7257,0.8359,0.9122,0.9581
0.2356,0.0549,0.1086,0.1918,0.3050,0.4401,0.5821,0.7136,0.8209,0.8983,0.9478


In [10]:
df4 = pd.DataFrame(puts, columns=['iv', 'spot', 'price', 'greek']).round(4)
df4['spot'] = df4.spot.astype(int)
df4_pivot = df4.pivot(index='iv', columns='spot', values='greek').round(4)
df4_pivot

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.1200,-0.9992,-0.9925,-0.9572,-0.8439,-0.6197,-0.3453,-0.1362,-0.0364,-0.0064,-0.0007
0.1344,-0.9975,-0.9850,-0.9375,-0.8163,-0.6069,-0.3609,-0.1634,-0.0546,-0.0132,-0.0023
0.1489,-0.9944,-0.9749,-0.9169,-0.7919,-0.5965,-0.3737,-0.1878,-0.0739,-0.0224,-0.0052
0.1633,-0.9896,-0.9628,-0.8964,-0.7705,-0.5878,-0.3842,-0.2094,-0.0934,-0.0336,-0.0097
0.1778,-0.9831,-0.9494,-0.8766,-0.7516,-0.5805,-0.3931,-0.2286,-0.1124,-0.0463,-0.0159
0.1922,-0.9751,-0.9351,-0.8578,-0.7349,-0.5742,-0.4007,-0.2456,-0.1307,-0.0598,-0.0235
0.2067,-0.9659,-0.9205,-0.8401,-0.7201,-0.5688,-0.4072,-0.2608,-0.1479,-0.0738,-0.0322
0.2211,-0.9558,-0.9059,-0.8236,-0.7069,-0.5640,-0.4129,-0.2743,-0.1641,-0.0878,-0.0419
0.2356,-0.9451,-0.8914,-0.8082,-0.6950,-0.5599,-0.4179,-0.2864,-0.1791,-0.1017,-0.0522


In [11]:
pvt = (df3_pivot + df4_pivot).round(4)
pvt.style.background_gradient(cmap='RdYlGn', axis=None)

spot,16660,16735,16811,16886,16962,17037,17113,17188,17264,17340
iv,,,,,,,,,,
0.12,-0.998400,-0.985000,-0.914400,-0.687800,-0.239400,0.309400,0.727600,0.927200,0.987200,0.998600
0.1344,-0.995000,-0.970000,-0.875000,-0.632600,-0.213800,0.278200,0.673200,0.890800,0.973600,0.995400
0.1489,-0.988800,-0.949800,-0.833800,-0.583800,-0.193000,0.252600,0.624400,0.852200,0.955200,0.989600
0.1633,-0.979200,-0.925600,-0.792800,-0.541000,-0.175600,0.231600,0.581200,0.813200,0.932800,0.980600
0.1778,-0.966200,-0.898800,-0.753200,-0.503200,-0.161000,0.213800,0.542800,0.775200,0.907400,0.968200
0.1922,-0.950200,-0.870200,-0.715600,-0.469800,-0.148400,0.198600,0.508800,0.738600,0.880400,0.953000
0.2067,-0.931800,-0.841000,-0.680200,-0.440200,-0.137600,0.185600,0.478400,0.704200,0.852400,0.935600
0.2211,-0.911600,-0.811800,-0.647200,-0.413800,-0.128000,0.174200,0.451400,0.671800,0.824400,0.916200
0.2356,-0.890200,-0.782800,-0.616400,-0.390000,-0.119800,0.164200,0.427200,0.641800,0.796600,0.895600
